In [1]:
import numpy as np
import pandas as pd
import sys
import os
import subprocess

import requests

import json

In [2]:
r = requests.get('https://financialmodelingprep.com/api/v3/historical-price-full/AAPL?serietype=line&apikey=b903b84d1ce1092d3e9fca34e25bf408')
a = r.json()
print(a)
print("SUCCESS")
print(pd.DataFrame(a))
print("Starting")


{'symbol': 'AAPL', 'historical': [{'date': '2021-04-16', 'close': 134.16}, {'date': '2021-04-15', 'close': 134.5}, {'date': '2021-04-14', 'close': 132.029999}, {'date': '2021-04-13', 'close': 134.429993}, {'date': '2021-04-12', 'close': 131.240005}, {'date': '2021-04-09', 'close': 133.0}, {'date': '2021-04-08', 'close': 130.360001}, {'date': '2021-04-07', 'close': 127.900002}, {'date': '2021-04-06', 'close': 126.209999}, {'date': '2021-04-05', 'close': 125.900002}, {'date': '2021-04-01', 'close': 123.0}, {'date': '2021-03-31', 'close': 122.150002}, {'date': '2021-03-30', 'close': 119.900002}, {'date': '2021-03-29', 'close': 121.389999}, {'date': '2021-03-26', 'close': 121.209999}, {'date': '2021-03-25', 'close': 120.589996}, {'date': '2021-03-24', 'close': 120.089996}, {'date': '2021-03-23', 'close': 122.540001}, {'date': '2021-03-22', 'close': 123.389999}, {'date': '2021-03-19', 'close': 119.989998}, {'date': '2021-03-18', 'close': 120.529999}, {'date': '2021-03-17', 'close': 124.7600

In [3]:
def getYLabels(priceDF):

    stock_data = priceDF

    names = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']

    print("Dimensions of file read: ")
    print("Rows")
    print(stock_data.shape[0])

    total_entries = stock_data.shape[0]


    #y-labels in following form:
    #(price_at_year_i, price_after_year_i, percentage change)

    #Initial time period = 1 year
    #On average 252 trading days so use current date + 252 entries to get approximate price one year later

    jsonYLabels = {}
    index = 1
    listDates = []
    listPriceOnDate = []
    listPriceAfterYear = []
    listPercentageChange = []
    for index2, dataPrice in stock_data.iterrows():
        if (index == 0):
            pass

        yLabelEntry = {}
        mapEntry = stock_data['historical'].iloc[index]
        current_date = mapEntry['date']
        current_price = float(mapEntry['close'])

        #Price a year later
        indexOneYearLater = index + 252
        #Check fence post to see if need to subtract 1 to avoid index out of range exception
        if (indexOneYearLater >= total_entries):
            break

        #If got here assume that indexOneYearLater not out of bounds
        mapEntryYearLater = stock_data['historical'].iloc[indexOneYearLater]
        price_after_year = float(mapEntryYearLater['close'])
        #price_after_year = float(stock_data['close'].iloc[indexOneYearLater])

        #print("Price after year")
        #print(price_after_year)

        #print("Current Price")
        #print(current_price)

        percentage_change = float((price_after_year - current_price)/current_price)
        #if (percentage_change < 0):
            #print("Negative percentage change")


        #Pass to json
        yLabelEntry['date'] = current_date
        yLabelEntry['price_on_date'] = current_price
        yLabelEntry['price_after_year'] = price_after_year
        yLabelEntry['percentage_change'] = percentage_change

        jsonYLabels[current_date] = yLabelEntry

        #Append to synchronous lists to create pandas dataframe
        listDates.append(current_date)
        listPriceOnDate.append(current_price)
        listPriceAfterYear.append(price_after_year)
        listPercentageChange.append(percentage_change)

        index = index + 1

        #check if new index (a year after) is below total_entries to see when to break

    #Create pandas table:
    d = {'date': listDates, 'price_on_date': listPriceOnDate, 'price_after_year': listPriceAfterYear, 'percentage_change': listPercentageChange}

    df = pd.DataFrame(data=d)

    return jsonYLabels, df


In [4]:
res = []
stockTickers = ['BFOR', 'REZI', 'BSCM', 'VOSO', 'CSR', 'SOGO', 'SRL', 'CD', 'MDXG', 'AAPL', 'MSFT', 'FB', 'TSLA', 'BLK', 'CZR', 'ANF', 'ARAY', 'AVAV', 'AGFY']
for ticker in stockTickers:
    requestString = 'https://financialmodelingprep.com/api/v3/historical-price-full/' + str(ticker) + '?serietype=line&apikey=b903b84d1ce1092d3e9fca34e25bf408'
    r = requests.get(requestString)
    a = r.json()
    dataFrameCur = pd.DataFrame(a)
    print("Stock Name: "+ str(ticker))
    print(dataFrameCur)
    print("")
    print("")
    json_y_labels, pd_y_labels = getYLabels(dataFrameCur)
    res.append(pd_y_labels)

Stock Name: BFOR
     symbol                                  historical
0      BFOR      {'date': '2021-04-16', 'close': 58.74}
1      BFOR  {'date': '2021-04-15', 'close': 58.279999}
2      BFOR  {'date': '2021-04-14', 'close': 57.799999}
3      BFOR      {'date': '2021-04-13', 'close': 57.82}
4      BFOR  {'date': '2021-04-12', 'close': 58.130001}
...     ...                                         ...
1977   BFOR  {'date': '2013-06-10', 'close': 25.120001}
1978   BFOR  {'date': '2013-06-07', 'close': 25.059999}
1979   BFOR      {'date': '2013-06-06', 'close': 24.68}
1980   BFOR       {'date': '2013-06-05', 'close': 24.6}
1981   BFOR  {'date': '2013-06-04', 'close': 24.879999}

[1982 rows x 2 columns]


Dimensions of file read: 
Rows
1982
Stock Name: REZI
    symbol                                  historical
0     REZI      {'date': '2021-04-16', 'close': 28.57}
1     REZI  {'date': '2021-04-15', 'close': 28.969999}
2     REZI      {'date': '2021-04-14', 'close': 28.59}
3     REZI 

In [5]:
combined = pd.concat(res)


In [6]:
print("Combined y labels: ")
print(combined)

Combined y labels: 
            date  price_on_date  price_after_year  percentage_change
0     2021-04-15      58.279999         32.880001          -0.435827
1     2021-04-14      57.799999         34.040001          -0.411073
2     2021-04-13      57.820000         33.330002          -0.423556
3     2021-04-12      58.130001         34.070000          -0.413900
4     2021-04-09      58.000000         33.250000          -0.426724
...          ...            ...               ...                ...
3326  2008-01-29      23.260000         22.879999          -0.016337
3327  2008-01-28      23.790001         23.160000          -0.026482
3328  2008-01-25      23.299999         23.600000           0.012876
3329  2008-01-24      23.049999         23.900000           0.036876
3330  2008-01-23      23.590000         23.930000           0.014413

[66408 rows x 4 columns]
